In [ ]:
import pandas as pd
import numpy as np
# Load data
md = pd.read_csv('../../data/lens_tmdb/movies_metadata.csv')
# ratings = pd.read_csv('../../data/lens_tmdb/ratings.csv')

# movies_metadata.csv

In [ ]:
md

In [ ]:
import pandas as pd
import ast

# assuming df is your DataFrame
# and assuming the dictionaries are stored as strings, we first need to convert them into actual Python objects
dict_columns = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']

for col in dict_columns:
    md[col] = md[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

# Now, let's extract the information we want
md['collection_name'] = md['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)
md['genre'] = md['genres'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 else np.nan)
md['production_company'] = md['production_companies'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 else np.nan)
md['production_country'] = md['production_countries'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 else np.nan)
md['spoken_language'] = md['spoken_languages'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 else np.nan)

# now drop the original columns
md = md.drop(columns=dict_columns)


In [ ]:
md

In [ ]:
md.columns

In [ ]:
# Save cleaned meta_data csv
md.to_csv('../../data/lens_tmdb/cleaned/movies_metadata_cleaned.csv', index=False)

In [ ]:
md

# credits.csv

In [ ]:
credits = pd.read_csv('../../data/lens_tmdb/credits.csv')

In [ ]:
credits

In [ ]:
import ast
# convert strings to lists of dictionaries
credits['cast'] = credits['cast'].apply(ast.literal_eval)
credits['crew'] = credits['crew'].apply(ast.literal_eval)

# extract the names of the cast members
credits['cast_names'] = credits['cast'].apply(lambda x: [i['name'] for i in x])

# extract the director from the crew column
credits['director'] = credits['crew'].apply(lambda x: [i['name'] for i in x if i['job'] == 'Director'])

# you can now drop the original 'cast' and 'crew' columns if you want
credits = credits.drop(columns=['cast', 'crew'])

credits.head()


In [ ]:
# Save cleaned credits csv
credits.to_csv('../../data/lens_tmdb/cleaned/credits_cleaned.csv', index=False)

# keywords.csv

In [ ]:
keywords = pd.read_csv('../../data/lens_tmdb/keywords.csv')

In [ ]:
keywords

In [ ]:
import pandas as pd
import ast

# convert strings to lists of dictionaries
keywords['keywords'] = keywords['keywords'].apply(ast.literal_eval)

# extract the keywords
keywords['keyword_list'] = keywords['keywords'].apply(lambda x: [i['name'] for i in x])

# you can now drop the original 'keywords' column if you want
keywords_df = keywords.drop(columns=['keywords'])

keywords_df.head()


In [ ]:
# Save cleaned keywords csv
keywords_df.to_csv('../../data/lens_tmdb/cleaned/keywords_cleaned.csv', index=False)

# create final movie dataframe

In [ ]:
md_cleaned = pd.read_csv('../../data/lens_tmdb/cleaned/movies_metadata_cleaned.csv')
credits_cleaned = pd.read_csv('../../data/lens_tmdb/cleaned/credits_cleaned.csv')
keywords_cleaned = pd.read_csv('../../data/lens_tmdb/cleaned/keywords_cleaned.csv')
ratings = pd.read_csv('../../data/lens_tmdb/ratings.csv')
ratings_small = pd.read_csv('../../data/lens_tmdb/ratings_small.csv')
links = pd.read_csv('../../data/lens_tmdb/links.csv')
links_small = pd.read_csv('../../data/lens_tmdb/links_small.csv')

# create dataset ***info*** from movies_metadata, credits, keywords

In [ ]:
md_cleaned.columns

In [ ]:
keywords_cleaned.columns

In [ ]:
credits_cleaned.columns

In [ ]:
md_cleaned['id'] = pd.to_numeric(md_cleaned['id'], errors='coerce')
md_cleaned = md_cleaned.dropna(subset=['id'])
md_cleaned['id'] = md_cleaned['id'].astype(int)


In [ ]:
# set id column to int
md_cleaned['id'] = md_cleaned['id'].astype(int)
credits_cleaned['id'] = credits_cleaned['id'].astype(int)

In [ ]:
# merge md_cleaned and credits_cleaned
info_df = pd.merge(md_cleaned, credits_cleaned, on='id')

In [ ]:
info_df.shape

In [ ]:
info_df.columns

In [ ]:
# merge info_df and keywords_cleaned
info_df = pd.merge(info_df, keywords_cleaned, on='id')

In [ ]:
info_df.shape

In [ ]:
# rename column 'tmdbId' to 'tmdb_id'
links =links.rename(columns={'tmdbId': 'id'})

In [ ]:
# drop nan values
links = links.dropna(subset=['id'])
# convert id column to int
links['id'] = links['id'].astype(int)

In [ ]:
links

In [ ]:
merged_df_final = info_df.merge(links, how='left', on='id')

In [ ]:
merged_df_final

In [ ]:
# check if nan in movieId column and how many
merged_df_final['movieId'].isnull().sum()

In [ ]:
# save merged_df_final to csv
merged_df_final.to_csv('../../data/lens_tmdb/cleaned/df_all.csv', index=False)